<h1 align="center">Training the Transformer Model</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Reunimos o [modelo Transformer completo](https://machinelearningmastery.com/joining-the-transformer-encoder-and-decoder-and-masking) e agora estamos prontos para treiná-lo para tradução automática neural. Usaremos um Dataset de treinamento para essa finalidade, que contém pares de frases curtas em `inglês` e `alemão`. Também revisitaremos o papel do mascaramento no cálculo das métricas de *accuracy* e **Loss** durante o processo de treinamento. 

Neste script, você aprenderá como treinar o modelo `Transformer` para **tradução automática neural**. Os pontos a estudar são: 

* Como preparar o conjunto de Dados de treinamento

* Como aplicar uma `máscara de preenchimento` (Padding Mask) aos cálculos de Loss e accuracy

* Como treinar o modelo Transformer.

# Preparando o conjunto de dados de treinamento

Para isso, você pode consultar um tutorial anterior que aborda o material sobre como [preparar os dados de texto](https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/) para treinamento. 

Você também usará um Dataset que contém pares de frases curtas em `Inglês` e `Alemão`, que você pode [baixar aqui](https://github.com/Rishav09/Neural-Machine-Translation-System/blob/master/english-german-both.pkl). Este Dataset específico já foi limpo removendo caracteres não-imprimíveis e não-alfabéticos e caracteres de pontuação, normalizando ainda mais todos os caracteres Unicode para `ASCII` e **alterando todas as letras maiúsculas para minúsculas**. Portanto, você pode pular a etapa de limpeza, que geralmente faz parte do processo de preparação de dados. No entanto, se você usar um conjunto de dados que não seja facilmente limpo, consulte o [tutorial anterior](https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/) para aprender como fazer isso.


Vamos prosseguir criando a classe `PrepareDataset` que implementa os seguintes passos:

* <font color="orange">Carrega o Dataset de um nome de arquivo especificado.</font>

In [1]:
# https://github.com/Rishav09/Neural-Machine-Translation-System

from pickle import load

clean_dataset = load(open('/home/eddygiusepe/24_Transformers_NLP/NLP_Transformers/Training_the_Transformer_Model/english-german-both.pkl', 'rb'))
clean_dataset

array([['i like both', 'ich mag beide'],
       ['she misses him', 'er fehlt ihr'],
       ['i followed him', 'ich folgte ihm'],
       ...,
       ['tom is cooking', 'tom kocht'],
       ['youre upset', 'sie sind besturzt'],
       ['do you see me', 'sehen sie mich']], dtype='<U370')

In [2]:
len(clean_dataset)

10000

In [3]:
clean_dataset[0]

array(['i like both', 'ich mag beide'], dtype='<U370')

* <font color="orange">Seleciona o número de sentenças a serem usadas no conjunto de dados. Como o conjunto de dados é grande, você reduzirá seu tamanho para limitar o tempo de treinamento. No entanto, você pode explorar usando o conjunto de dados completo como uma extensão deste tutorial.</font>

```
dataset = clean_dataset[:self.n_sentences, :]
```

* <font color="orange">Acrescenta tokens de início (`<START>`) e end-of-string - fim de string  (`<EOS>`) a cada sentença. Por exemplo, a frase em inglês, `i like to run`, agora se torna, `<START> i like to run <EOS>`. Isso também se aplica à sua tradução correspondente em Alemão, `ich gehe gerne joggen`, que agora se torna, `<START> ich gehe gerne joggen <EOS>`.</font>

```
for i in range(dataset[:, 0].size):
	dataset[i, 0] = "<START> " + dataset[i, 0] + " <EOS>"
	dataset[i, 1] = "<START> " + dataset[i, 1] + " <EOS>"
```


* <font color="orange">Embaralha o conjunto de Dados aleatoriamente.</font>

```
shuffle(dataset)
```

* <font color="orange">Divide o conjunto de Dados embaralhado com base em uma proporção predefinida.</font>

```
train = dataset[:int(self.n_sentences * self.train_split)]
```

* <font color="orange">Cria e treina um tokenizador nas sequências de texto que serão alimentadas no codificador (Encoder) e localiza o comprimento da sequência mais longa, bem como o tamanho do vocabulário. </font>

```
enc_tokenizer = self.create_tokenizer(train[:, 0])
enc_seq_length = self.find_seq_length(train[:, 0])
enc_vocab_size = self.find_vocab_size(enc_tokenizer, train[:, 0])
```

* <font color="orange">Tokeniza as sequências de texto que serão alimentadas no codificador (Encoder), criando um vocabulário de palavras e substituindo cada palavra por seu índice de vocabulário correspondente. Os tokens `<START>` e `<EOS>` também farão parte deste vocabulário. Cada sequência também é preenchida (Padded) até o comprimento máximo da frase. </font>

```
trainX = enc_tokenizer.texts_to_sequences(train[:, 0])
trainX = pad_sequences(trainX, maxlen=enc_seq_length, padding='post')
trainX = convert_to_tensor(trainX, dtype=int64)
```

* <font color="orange">Cria e treina um tokenizador nas sequências de texto que serão alimentadas no decodificador (Decoder) e encontra o comprimento da sequência mais longa, bem como o tamanho do vocabulário.</font>

```
dec_tokenizer = self.create_tokenizer(train[:, 1])
dec_seq_length = self.find_seq_length(train[:, 1])
dec_vocab_size = self.find_vocab_size(dec_tokenizer, train[:, 1])
```

* <font color="orange">Repete um procedimento semelhante de tokenização e preenchimento para as sequências de texto que serão alimentadas no decodificador.</font>

```
trainY = dec_tokenizer.texts_to_sequences(train[:, 1])
trainY = pad_sequences(trainY, maxlen=dec_seq_length, padding='post')
trainY = convert_to_tensor(trainY, dtype=int64)
```

A listagem completa do código é a seguinte (consulte [este tutorial anterior](https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/) para mais detalhes):




In [5]:
from pickle import load
from numpy.random import shuffle
from keras.preprocessing.text import Tokenizer

#from tensorflow.keras.preprocessing.sequence import pad_sequences # Isto tambén funciona
from keras.utils.data_utils import pad_sequences # Isto funciona sem erro

from tensorflow import convert_to_tensor, int64

In [6]:
class PrepareDataset:
	def __init__(self, **kwargs):
		super(PrepareDataset, self).__init__(**kwargs)
		self.n_sentences = 10000  # Number of sentences to include in the dataset
		self.train_split = 0.9  # Ratio of the training data split
 
	# Fit a tokenizer
	def create_tokenizer(self, dataset):
		tokenizer = Tokenizer()
		tokenizer.fit_on_texts(dataset)
 
		return tokenizer
 
	def find_seq_length(self, dataset):
		return max(len(seq.split()) for seq in dataset)
 
	def find_vocab_size(self, tokenizer, dataset):
		tokenizer.fit_on_texts(dataset)
 
		return len(tokenizer.word_index) + 1
 
	def __call__(self, filename, **kwargs):
		# Load a clean dataset
		clean_dataset = load(open(filename, 'rb'))
 
		# Reduce dataset size
		dataset = clean_dataset[:self.n_sentences, :]
 
		# Include start and end of string tokens
		for i in range(dataset[:, 0].size):
			dataset[i, 0] = "<START> " + dataset[i, 0] + " <EOS>"
			dataset[i, 1] = "<START> " + dataset[i, 1] + " <EOS>"
 
		# Random shuffle the dataset
		shuffle(dataset)
 
		# Split the dataset
		train = dataset[:int(self.n_sentences * self.train_split)]
 
		# Prepare tokenizer for the encoder input
		enc_tokenizer = self.create_tokenizer(train[:, 0])
		enc_seq_length = self.find_seq_length(train[:, 0])
		enc_vocab_size = self.find_vocab_size(enc_tokenizer, train[:, 0])
 
		# Encode and pad the input sequences
		trainX = enc_tokenizer.texts_to_sequences(train[:, 0])
		trainX = pad_sequences(trainX, maxlen=enc_seq_length, padding='post')
		trainX = convert_to_tensor(trainX, dtype=int64)
 
		# Prepare tokenizer for the decoder input
		dec_tokenizer = self.create_tokenizer(train[:, 1])
		dec_seq_length = self.find_seq_length(train[:, 1])
		dec_vocab_size = self.find_vocab_size(dec_tokenizer, train[:, 1])
 
		# Encode and pad the input sequences
		trainY = dec_tokenizer.texts_to_sequences(train[:, 1])
		trainY = pad_sequences(trainY, maxlen=dec_seq_length, padding='post')
		trainY = convert_to_tensor(trainY, dtype=int64)
 
		return trainX, trainY, train, enc_seq_length, dec_seq_length, enc_vocab_size, dec_vocab_size

Antes de continuar treinando o modelo do Transformer, vamos primeiro dar uma olhada na saída da classe `PrepareDataset` correspondente à primeira frase no Dataset de Treinamento:

In [7]:
# Prepare the training data
dataset = PrepareDataset()

trainX, trainY, train_orig, enc_seq_length, dec_seq_length, enc_vocab_size, dec_vocab_size = dataset('english-german-both.pkl')
 
print(train_orig[0, 0], '\n', trainX[0, :])


# Observação: como o Dataset foi embaralhado aleatoriamente, você provavelmente verá uma saída diferente.

2022-11-17 20:29:37.187925: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-17 20:29:37.187986: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eddygiusepe): /proc/driver/nvidia/version does not exist
2022-11-17 20:29:37.188751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<START> tom spoke <EOS> 
 tf.Tensor([  1   4 697   2   0   0   0], shape=(7,), dtype=int64)


Você pode ver que, originalmente, você tinha uma frase de três palavras (`did tom tell you`) à qual você anexou os tokens de início e fim da string. Em seguida, você começou a vetorizar (você pode notar que os tokens `<START>` e `<EOS>` são atribuídos aos índices de vocabulário $1$ e $2$, respectivamente). O texto vetorizado também foi preenchido (padded) com zeros, de forma que o comprimento do resultado final corresponda ao comprimento máximo da sequência do codificador:

In [8]:
print('Encoder sequence length:', enc_seq_length)

Encoder sequence length: 7


Da mesma forma, você pode verificar os dados de destino (target) correspondentes que são alimentados no decodificador (Decoder):

In [9]:
print(train_orig[0, 1], '\n', trainY[0, :])

<START> tom sprach <EOS> 
 tf.Tensor([  1   5 783   2   0   0   0   0   0   0   0   0], shape=(12,), dtype=int64)


Aqui, o comprimento do resultado final corresponde ao comprimento máximo da sequência do decodificador (Decoder):

In [10]:
print('Decoder sequence length:', dec_seq_length)

Decoder sequence length: 12


# Aplicando uma máscara de preenchimento (Padding Mask) aos cálculos da Loss e Accuracy

[Lembre-se](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras) de que a importância de ter um padding mask no codificador (ENCODER) e no decodificador (DECODER) é garantir que os **valores zero** que acabamos de anexar às entradas vetorizadas não sejam processados ​​junto com os valores de entrada reais. 

Isso também vale para o processo de treinamento, onde uma máscara de preenchimento (padding mask) é necessária para que os valores de preenchimento (padding) zero nos dados de destino não sejam considerados no cálculo da Loss e Accuracy.

Vamos dar uma olhada no cálculo da `Loss` primeiro. 

Isso será calculado usando uma função de `Loss de entropia cruzada categórica esparsa` (**sparse categorical cross-entropy loss**) entre os valores de destino e preditos e subsequentemente multiplicado por uma máscara de preenchimento para que apenas os valores diferentes de zero válidos sejam considerados. A perda retornada é a média dos valores não mascarados:

In [11]:
def loss_fcn(target, prediction):
    # Create mask so that the zero padding values are not included in the computation of loss
    padding_mask = math.logical_not(equal(target, 0))
    padding_mask = cast(padding_mask, float32)
 
    # Compute a sparse categorical cross-entropy loss on the unmasked values
    loss = sparse_categorical_crossentropy(target, prediction, from_logits=True) * padding_mask
 
    # Compute the mean loss over the unmasked values
    return reduce_sum(loss) / reduce_sum(padding_mask)

Para o cálculo da `accuracy`, os valores `previstos` e `alvo` (target) são primeiro comparados. A saída prevista é um tensor de tamanho (`batch_size`, `dec_seq_length`, `dec_vocab_size`) e contém valores de probabilidade (gerados pela função `softmax` no lado do decodificador (Decoder)) para os tokens na saída. Para poder realizar a comparação com os valores alvo, considera-se apenas cada token com o maior valor de probabilidade, sendo recuperado o seu índice de dicionário através da operação: `argmax(prediction, axis=2)`. Após a aplicação de uma máscara de preenchimento (`Padding Mask`), a accuracy retornada é a média dos valores não mascarados:

In [12]:

def accuracy_fcn(target, prediction):
    # Create mask so that the zero padding values are not included in the computation of accuracy
    padding_mask = math.logical_not(math.equal(target, 0))
 
    # Find equal prediction and target values, and apply the padding mask
    accuracy = equal(target, argmax(prediction, axis=2))
    accuracy = math.logical_and(padding_mask, accuracy)
 
    # Cast the True/False values to 32-bit-precision floating-point numbers
    padding_mask = cast(padding_mask, float32)
    accuracy = cast(accuracy, float32)
 
    # Compute the mean accuracy over the unmasked values
    return reduce_sum(accuracy) / reduce_sum(padding_mask)

# Treinando o modelo do Transformer

In [13]:
# Define the model parameters
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_model = 512  # Dimensionality of model layers' outputs
d_ff = 2048  # Dimensionality of the inner fully connected layer
n = 6  # Number of layers in the encoder stack
 

# Define the training parameters
epochs = 2
batch_size = 64
beta_1 = 0.9
beta_2 = 0.98
epsilon = 1e-9
dropout_rate = 0.1


# Observação: 
# considere apenas duas épocas para limitar o tempo de treinamento. No entanto, você pode explorar o treinamento do modelo mais como uma extensão deste tutorial.

Você também precisa implementar um `escalonador de taxa de aprendizado` (Learning rate scheduler) que inicialmente aumente a taxa de aprendizado linearmente para os primeiros `warmup_steps` e, em seguida, diminua-a proporcionalmente à raiz quadrada inversa do número da passos (step number). `Vaswani et al.` expresse isso pela seguinte fórmula:

$$
learning\_rate = d\_model^{-0.5}.min(step^{-0.5}, step.warmup\_steps^{-0.5})
$$

In [16]:
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow import data, train, math, reduce_sum, cast, equal, argmax, float32, GradientTape, TensorSpec, function, int64



class LRScheduler(LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, **kwargs):
        super(LRScheduler, self).__init__(**kwargs)
 
        self.d_model = cast(d_model, float32)
        self.warmup_steps = warmup_steps
 
    def __call__(self, step_num):
 
        # Linearly increasing the learning rate for the first warmup_steps, and decreasing it thereafter
        arg1 = step_num ** -0.5
        arg2 = step_num * (self.warmup_steps ** -1.5)
 
        return (self.d_model ** -0.5) * math.minimum(arg1, arg2)
        

Uma instância da classe `LRScheduler` é subsequentemente transmitida como o argumento `learning_rate` do otimizador `Adam`: